LOAD DATASET

In [1]:
%pwd

'/content'

In [ ]:
import os
os.chdir('../')

In [1]:
!pip install langchain-community

In [6]:
!pip install langchain langchain-core langchain-community pandas
!pip install langchain-core pandas

INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 449.8/449.8 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 57.1 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 1.0.0
    Uninstalling langchain-core-1.0.0:
      Successfully uninstalled langchain-core-1.0.0
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 1.0.0
    Uninstalling langchain-text-splitters-1.0.0:
      Successfully uninstalled langchain-text-splitters-1.0.0
  Attempting uninstall: langchain-community
    Found existing installation: langchain-community 0.4
    Uninstalling langchain-community-0.4:
      Successfully uninstalled langchain-community-0.4
ERROR: pip's dependency resolver does not currently take into account all the pa

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
^C


In [1]:
import pandas as pd
from langchain.schema import Document

def load_csv_file(file_path):
    # Đọc file CSV
    df = pd.read_csv(file_path)

    documents = []
    for _, row in df.iterrows():
        # Kết hợp các trường thành nội dung văn bản
        content = f"Tiêu đề: {row['title']}\nNgày đăng: {row['date']}\nNội dung: {row['content']}"

        # Tạo metadata từ các trường còn lại
        metadata = {
            'link': row['link'],
            'image': row['image'],
            'push_date': row['push_date'],
            'id_news': row['id_news'],
            'detail_images': row['detail_images']
        }

        # Tạo Document object
        doc = Document(page_content=content, metadata=metadata)
        documents.append(doc)

    return documents

# Sử dụng hàm
extracted_data = load_csv_file('/content/dalieu_articles.csv')

# In ra để kiểm tra
print(f"Đã tải được {len(extracted_data)} documents")
if extracted_data:
    print("\nDocument đầu tiên:")
    print(extracted_data[0].page_content[:500] + "...")
    print("\nMetadata:", extracted_data[0].metadata)

Đã tải được 390 documents

Document đầu tiên:
Tiêu đề: U mạch sừng hóa (Angiokeratoma)
Ngày đăng: Ngày đăng: 29/09/2025
Nội dung: 1. Đại cương
U mạch sừng hóa là một tổn thương mạch máu lành tính, đặc trưng bởi sự giãn mao mạch ở lớp trung bì nông kèm tăng lớp sừng. Về lâm sàng, tổn thương thường xuất hiện dưới dạng sẩn hoặc mảng màu sắc thay đổi từ đỏ, tím đến đen, bề mặt gồ ghề, có thể đơn độc hoặc đa ổ, dễ chảy máu khi gặp chấn thương.
Nguyên nhân được cho là do sự suy yếu của thành mạch máu, có thể là bẩm sinh hoặc mắc phải khiến các ma...

Metadata: {'link': 'https://dalieu.vn/u-mach-sung-hoa-angiokeratoma-d5287.html', 'image': 'https://admin-dalieu.cdn.vccloud.vn/files/uploads/600/u-mach-sung-hoa-angiokeratoma-143540.jpg', 'push_date': '2025-09-29 14:35:00', 'id_news': 5287, 'detail_images': 'https://admin.dalieu.vn/image.php?file=https://admin.dalieu.vn/files/uploads/2025/09/1-1431.jpg; https://admin.dalieu.vn/image.php?file=https://admin.dalieu.vn/files/uploads/2025/09/2-1431.j

Chunking


In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=512,
        chunk_overlap=20,
        separators=["\n\n", "\n", ". ", " ", ""]
    )
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [5]:
text_chunks = text_split(extracted_data)
print(f'So luong chunk da tach', len(text_chunks))

So luong chunk da tach 7378


MODEL EMBEDDINGS

In [6]:
from langchain.embeddings import HuggingFaceEmbeddings


def download_huggingface_embeddings():
    embeddings = HuggingFaceEmbeddings(
        model_name="dangvantuan/vietnamese-embedding"
    )
    return embeddings

In [7]:
embeddings = download_huggingface_embeddings()

/tmp/ipython-input-4007518188.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/171 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

bpe.codes: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/22.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/965 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [8]:
query_result = embeddings.embed_documents(["Hà Nội là thủ đô của Việt Nam", "Đà Nẵng là thành phố du lịch"])
print("Chiều của vector 1:", len(query_result[0])) #- > thấy là dù cho vui_tính có dấu _ SBERT này vẫn xử lý tốt 768 embeddings đúng


Chiều của vector 1: 768


LOAD KEY

In [9]:
from dotenv import load_dotenv
load_dotenv()

False

In [10]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
OPENAI_API_KEY=os.environ.get('DEEPSEEK_API_KEY')

NameError: name 'os' is not defined